# Convert Pytorch Model into Tensorflow Model and Keras

Ref: https://github.com/aobo-y/hair-dye/blob/master/src/torch2tf.ipynb

In this notebook, my environment is as follows, torch version is 0.4.1.  


`pip install tensorflow==1.13.1 onnx==1.2.1 onnx-tf==1.1.2`


make sure the version. some version may cause Upsample version 1 is not implemented problem.
If that happens try to roll back to older versions

In [1]:
import os
import tensorflow as tf
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import onnx
from onnx_tf.backend import prepare
#from onnx import version_converter, helper

/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: P

/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






In [2]:
from models import *

In [3]:
print(tf.__version__)
print(onnx.__version__)

1.14.0
1.6.0


# configurations

In [4]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')
DIR_PATH = os.path.abspath('') # current path
SAVE_PATH = os.path.join(DIR_PATH, "models/state_dicts/")
TORCH_MODEL_PATH = os.path.join(SAVE_PATH, "resnet18.pt")
ONNX_MODEL_PATH = os.path.join(SAVE_PATH, "resnet18.onnx")
TF_MODEL_PATH = os.path.join(SAVE_PATH, "resnet18.pb")


In [5]:
USE_CUDA

True

In [6]:
print(device)

cuda


In [7]:
print(TORCH_MODEL_PATH)

/workspace/DeepConcolic/models/state_dicts/resnet18.pt


# load torch model and export it to ONNX model

In [8]:
def build_model(path, device):
    model = resnet18(pretrained=True)
#     checkpoint = torch.load(path, map_location=device)
#     model.load_state_dict(checkpoint)
    # use appropriate device
#     model = model.to(device)
    
    return model


In [9]:
torch_model = build_model(TORCH_MODEL_PATH, device)
# export model to ONNX
# ONNX need a dummy input
dummy_input = Variable(torch.randn(1, 3, 32, 32))
# pred = torch_model(dummy_input)
# print("output size: ", pred.size())

input_names = ["input"]
output_names = ["output"]
torch.onnx.export(torch_model, dummy_input, ONNX_MODEL_PATH, verbose=True, input_names=input_names, output_names=output_names)

Run using pretrained model
graph(%input : Float(1:3072, 3:1024, 32:32, 32:1, requires_grad=0, device=cpu),
      %fc.weight : Float(10:512, 512:1, requires_grad=1, device=cpu),
      %fc.bias : Float(10:1, requires_grad=1, device=cpu),
      %200 : Float(64:27, 3:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %201 : Float(64:1, requires_grad=0, device=cpu),
      %203 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %204 : Float(64:1, requires_grad=0, device=cpu),
      %206 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %207 : Float(64:1, requires_grad=0, device=cpu),
      %209 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %210 : Float(64:1, requires_grad=0, device=cpu),
      %212 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %213 : Float(64:1, requires_grad=0, device=cpu),
      %215 : Float(128:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %216 : Float(128:1, requires_grad=0, device=cp

# Import the ONNX model to tensorflow

In [10]:
# load the onnx file
onnx_model = onnx.load(ONNX_MODEL_PATH)
# Check the model
onnx.checker.check_model(onnx_model)
# print('The model before conversion:\n{}'.format(onnx_or_model))

# # A full list of supported adapters can be found here:
# # https://github.com/onnx/onnx/blob/master/onnx/version_converter.py#L21
# # Apply the version conversion on the original model
# onnx_model = version_converter.convert_version(onnx_or_model, 6)

# print('The model after conversion:\n{}'.format(onnx_model))


In [11]:
# import onnx to TF model
tf_rep = prepare(onnx_model)

/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of GatherND in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/root/anaconda3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageS

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.


/root/anaconda3/lib/python3.7/site-packages/onnx_tf/handlers/backend/pool_mixin.py:209: UserWarning: Using the pooling op in compatibility mode.This means your graph cannot be serialized.Please configure your pooling operation to only use paddings that correspond to Tensorflow SAME or VALID padding.
  "correspond to Tensorflow SAME or VALID padding.", UserWarning)


# Save to file

In [12]:
tf_rep.export_graph(TF_MODEL_PATH)

# Run the model to see if it works

In [13]:
class ImgTransformer():
    def __init__(self, color_aug=False):
        self.color_aug = color_aug

    def transform(self, image):
        transformer = transforms.Compose([
            transforms.ToTensor()        
        ])

        transform_image = transformer(image)

        return transform_image
#         return image

    def load(self, path):
        image = Image.open(path).convert('RGB')
        return self.transform(image)

In [14]:
def create_multi_figure(rows, dye=False):
    fig = plt.figure()

  # 3 tensors, the middle one is mask
    if len(rows[0]) == 3:
        names = ["Image", "Mask", "Prediction"]
    else:
        names = ["Image", "Prediction"]

    for i, data in enumerate(rows):
        img = data[0]
        prediction = data[-1]

        # reverse normalization of the oriignal image
        img = (img + 1) / 2
        data[0] = img

        if dye:
            transform_hue = transforms.Compose([
            transforms.ToTensor()
          ])
            dyed = transform_hue(img)

            dyed = prediction * dyed + (1 - prediction) * img
            data.append(dyed)
            names.append('Dye')


        for j, d in enumerate(data):
            d = d.squeeze()
            im = d.data.numpy()

            if im.shape[0] != 3:
                im = np.expand_dims(im, axis=0)
                im = np.concatenate((im, im, im), axis=0)

            im = im.transpose(1, 2, 0)

            f = fig.add_subplot(len(rows), len(data), i * len(data)+ j + 1)
            f.imshow(im)
            if i == 0:
                f.set_title(names[j])
                f.set_xticks([])
                f.set_yticks([])

    return fig

In [15]:
img_path = os.path.join(DIR_PATH, "images", "airplane5.png")
transformer = ImgTransformer(color_aug=False)
img =  transformer.load(img_path)

In [16]:
display(img.numpy())

array([[[0.8352941 , 0.8352941 , 0.8392157 , ..., 0.8117647 ,
         0.8117647 , 0.8117647 ],
        [0.8509804 , 0.84705883, 0.8745098 , ..., 0.8235294 ,
         0.8235294 , 0.81960785],
        [0.8509804 , 0.85882354, 0.92941177, ..., 0.827451  ,
         0.827451  , 0.81960785],
        ...,
        [0.9098039 , 0.8980392 , 0.8980392 , ..., 0.8901961 ,
         0.8901961 , 0.8901961 ],
        [0.9098039 , 0.8980392 , 0.9019608 , ..., 0.89411765,
         0.8901961 , 0.8901961 ],
        [0.90588236, 0.89411765, 0.8980392 , ..., 0.8862745 ,
         0.8862745 , 0.8862745 ]],

       [[0.92941177, 0.9137255 , 0.9137255 , ..., 0.91764706,
         0.91764706, 0.91764706],
        [0.94509804, 0.92156863, 0.9411765 , ..., 0.92941177,
         0.92941177, 0.9254902 ],
        [0.94509804, 0.93333334, 0.98039216, ..., 0.93333334,
         0.93333334, 0.9254902 ],
        ...,
        [0.98039216, 0.96862745, 0.96862745, ..., 0.9647059 ,
         0.9647059 , 0.9647059 ],
        [0.9

In [17]:
def evaluateOne(img, model, absolute=True):
    img = img.to(device).unsqueeze(0)
    pred = model.run(img.cpu())[0]
    pred = torch.from_numpy(pred)
    if absolute:
        pred[pred > .5] = 1.
        pred[pred <= .5] = 0.
    else:
        pred[pred < .4] = 0

    rows = [[img[0], pred[0]]]
#     create_multi_figure(rows, dye=True)
    print(pred[0])
#     plt.show()


In [18]:
evaluateOne(img, tf_rep)


tensor([1., 0., 0., 1., 0., 1., 0., 0., 0., 0.])
